In [1]:
import os
import numpy as np
import fitsio
from astropy.cosmology import FlatLambdaCDM

The below code should be run at NERSC to generate a himalayas catalog usable for voidfinding

In [2]:
#location of himalayas tiles
directory_a = "/global/cfs/cdirs/desi/spectro/redux/himalayas/tiles/cumulative"


In [3]:
#matter fraction
omega_M=0.315
#create cosmology
csm0 = FlatLambdaCDM(H0=100, Om0=omega_M)

In [4]:
#converts flux in nanomaggies to a magnitude
f_to_m = lambda f: 22.5-2.5*np.log10(f)


In [5]:
#the maximum redshift of himalayas galaxies to be selected
zlim = 0.24

In [7]:
#the path to the output for the error log
#the error log saves where in the himalayas tiles the program terminated,
#allowing future attempts to resume in the correct location
error_log="error_log.txt"

In [52]:
print(len(os.listdir(directory_a)))

6167


In [55]:
#the path to the output for the himilayas catalog
himalayas="himalayas.txt"

In [56]:
try:
    #remove previous attempt
    #in final version, this will be optional
    if os.path.exists(himalayas):
        os.remove(himalayas)
    #create output file
    np.savetxt(himalayas,[],header="ra dec redshift rabsmag",fmt='%s')
    #loop through himilayas folders and files
    #in final version, teh slice will be removed
    for directory_b in os.listdir(directory_a):
        for directory_c in os.listdir(f"{directory_a}/{directory_b}"):
            directory=f"{directory_a}/{directory_b}/{directory_c}"
            #select redrock files
            for file in os.listdir (directory):
                if len(file)>=7 and file[:7]=="redrock":
                    
                    #open the current redrock file
                    with fitsio.FITS(f'{directory}/{file}') as fits:
                        
                        redshifts = fits[1]['TARGETID','Z','ZWARN','DELTACHI2','SPECTYPE'][:]
                        fibermap = fits[2]['TARGETID','TARGET_RA','TARGET_DEC','BGS_TARGET','FLUX_R'][:]
                        #ensure that targetids are matched
                        if np.array_equal(redshifts["TARGETID"],fibermap["TARGETID"]):
                            pass
                        else:
                            raise Exception("redrock redshift and fibermap tables do not have matching TARGET_ID values")
                        #cuts on redshift range, redshift fit quality, 
                        #galaxies only, bgs bright
                        select = np.where((redshifts['Z']>0) 
                                           & (redshifts['Z']<=zlim) 
                                           & (redshifts['ZWARN']==0) 
                                           & (redshifts['DELTACHI2']>30) 
                                           & (redshifts['SPECTYPE']=='GALAXY') 
                                           & (fibermap['BGS_TARGET'] & 2**1 != 0))
                        
                        fibermap = fibermap[['TARGET_RA','TARGET_DEC','FLUX_R']][select]
                        redshifts=redshifts['Z'][select]

                        magMr = -np.array(csm0.distmod(redshifts)) + f_to_m(fibermap['FLUX_R'])

                        #magnitude cut
                        select = (magMr<-19.5)
                        fibermap=fibermap[select]
                        redshifts=redshifts[select]
                        magMr=magMr[select]

                        out=np.rec.fromarrays([fibermap['TARGET_RA'],
                                               fibermap['TARGET_DEC'],
                                               redshifts,
                                               magMr], 
                                              dtype=[('ra',fibermap['TARGET_RA'].dtype),
                                                     ('dec',fibermap['TARGET_DEC'].dtype),
                                                     ('redshift',redshifts.dtype),
                                                     ('rabsmag',magMr.dtype)])

                        with open(himalayas,'ab') as f:
                            np.savetxt(f,out,fmt='%s')
except Exception as e:
    with open(error_log,'a') as f:
        f.write(f'Program terminated while in {directory}/{file}')
        f.write('\n')
        #the below line can caluse errors, replace with logging
        #f.write(e.message)
        #f.write('\n')